In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow
import tensorflow as tf
import cv2
import numpy
import matplotlib.pyplot as plt
import glob
import PIL
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model

In [0]:
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
classifier = Sequential()
model = Sequential()

In [0]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1)
top3_acc.__name__ = 'top3_acc'
top1_acc.__name__ = 'top1_acc'

In [6]:
data = []
label = []
def load_image(name,index):
    path = "/content/drive/My Drive/dataset/" + name + "/*JPEG"
    for file in glob.glob(path):
        img = cv2.imread(file) 
        img  = cv2.resize(img,(224,224))
        data.append(img)
        label.append(index)
    return data,label
i=0 
path = "dataset/"
dog_name = ["bullmastiff","chowchow","pug","maltese","huskysibir","dachshund","dalmatian","corgi","chihuahua","yorkshire"]
for name in dog_name:
    load_image(name,i)
    print ("loading", i)
    i +=1

loading 0
loading 1
loading 2
loading 3
loading 4
loading 5
loading 6
loading 7
loading 8
loading 9


In [0]:
label = tf.keras.utils.to_categorical(label,num_classes=10)

In [0]:
(trainX, testX, trainY, testY) = train_test_split(numpy.array(data),
	numpy.array(label), test_size=0.25)

In [13]:
vgg = keras.applications.vgg16.VGG16(include_top=True,weights="imagenet")
for layer in vgg.layers:
  layer.trainable = False

softmax = Dense(10,activation='softmax')(vgg.layers[-2].output)

model = Model(vgg.input,softmax)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
print("[INFO] training network...")
opt = Adam(lr=1e-4, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=['top_k_categorical_accuracy',top1_acc,top3_acc])
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=5, batch_size=32)

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Train on 7279 samples, validate on 2427 samples
Epoch 1/5
7279/7279 [==============================] - 96s 13ms/step - loss: 0.9557 - top_k_categorical_accuracy: 0.9389 - top1_acc: 0.7089 - top3_acc: 0.8773 - val_loss: 0.4284 - val_top_k_categorical_accuracy: 0.9909 - val_top1_acc: 0.8537 - val_top3_acc: 0.9699
Epoch 2/5
7279/7279 [==============================] - 87s 12ms/step - loss: 0.3600 - top_k_categorical_accuracy: 0.9930 - top1_acc: 0.8801 - top3_acc: 0.9747 - val_loss: 0.3528 - val_top_k_categorical_accuracy: 0.9938 - val_top1_acc: 0.8793 - val_top3_acc: 0.9769
Epoch 3/5
7279/7279 [==============================] - 87s 12ms/step - loss: 0.2852 - top_k_categorical_accuracy: 0.9963 - top1_acc: 0.9025 - top3_acc: 0.9841 - val_loss: 0.3319 - val_top_k_categorical_accuracy: 0.9930 - val_top1_acc: 0.8842 - val_top3_acc: 0.9786
Epoch 4/5
7279/7279 [==============================] - 87s 12ms/step - loss: 0.239